In [1]:
# Input data processing
import pandas as pd
import numpy as np
df = pd.read_table('iris.data', header=None)
raw_data = np.array(df)
pro_data = [[],[],[]]
for item in raw_data:
    pro_item = str(item[0]).split(',')
    pro_item = [eval(pro_item[0]),eval(pro_item[1]),eval(pro_item[2]),eval(pro_item[3]),pro_item[4]]
    if pro_item[4] == 'Iris-setosa':    
        pro_item[4] = 0
        pro_data[0].append(pro_item)
    if pro_item[4] == 'Iris-versicolor': 
        pro_item[4] = 1
        pro_data[1].append(pro_item)
    if pro_item[4] == 'Iris-virginica':  
        pro_item[4] = 2
        pro_data[2].append(pro_item)
pro_data = np.array(pro_data)
train_ratio = 0.8
def train_test(all_data, train_ratio):
    train_len = round(len(all_data)*train_ratio)
    np.random.shuffle(all_data)
    train_data = all_data[0:train_len]
    test_data = all_data[train_len:len(all_data)]
    return[train_data,test_data]
# print(train_test(pro_data[0], train_ratio))
m = 4 # attribute
n = 3 # types
# save training and testing data
training_set = [[[] for col in range(m)] for row in range(n)]
testing_set = [[[] for col in range(m)] for row in range(n)]
# for each type
for ii in range(n):
    x = train_test(pro_data[ii],train_ratio)
#     for each attribute
    for jj in range(m):
        training_set[ii][jj] = x[0][:,jj]
        testing_set[ii][jj]=x[1][:,jj]

In [2]:
# EM iteration to estimate GMM coefficients with training data set
from sklearn.mixture import GaussianMixture
# GMM coefficients
components_num = 3
weights = [[[[] for item in range(components_num)] for col in range(m)] for row in range(n)]
means = [[[[] for item in range(components_num)] for col in range(m)] for row in range(n)]
covariances = [[[[] for item in range(components_num)] for col in range(m)] for row in range(n)]
for ii in range(n):
    for jj in range(m):
        gmm_coefficients = GaussianMixture(n_components=3).fit(training_set[ii][jj].reshape(-1, 1))
        for item in range(components_num):
            weights[ii][jj][item] = gmm_coefficients.weights_[item]
            means[ii][jj][item] = gmm_coefficients.means_[item][0]
            covariances[ii][jj][item] = gmm_coefficients.covariances_[item][0]

c:\Users\29265\.conda\envs\quenv\lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
c:\Users\29265\.conda\envs\quenv\lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
c:\Users\29265\.conda\envs\quenv\lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
c:\Users\29265\.conda\envs\quenv\lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known

In [3]:
# Plot GMM curves


1

In [4]:
# Testing


In [5]:
# Generate BPAs


In [6]:
# Combining BPAs on quantum circuits and conduct measurement


In [7]:
# Decision-making by the combined BPA
